In [5]:
import numpy as np
import pandas as pd
import models
import warnings
warnings.simplefilter('ignore')
import math
from sklearn.pipeline import Pipeline
from sklearn import set_config
from matplotlib import style
from matplotlib.patches import Ellipse
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE

In [6]:
hankel = models.Hankel()
rank = models.Rank()
cluster = models.Cluster()
meepc = models.MEEPC() 

In [7]:
df = pd.read_csv('~/data/training_data.csv')

In [8]:
df

,1_AIT_001_PV,1_AIT_002_PV,1_AIT_003_PV,1_AIT_004_PV,1_AIT_005_PV,1_FIT_001_PV,1_LT_001_PV,1_MV_001_STATUS,1_MV_002_STATUS,1_MV_003_STATUS,...,3_AIT_001_PV,3_AIT_002_PV,3_AIT_003_PV,3_AIT_004_PV,3_AIT_005_PV,3_FIT_001_PV,3_LT_001_PV,LEAK_DIFF_PRESSURE,TOTAL_CONS_REQUIRED_FLOW,"Attack LABLE (1:No Attack, -1:Attack)"
0,167.892,0.625471,11.6175,504.140,0.293168,0.001107,62.1436,1.0,1.0,1.0,...,0.0,8279.1,14.35530,238.111,0.620147,0.667695,67.0974,60.0435,0.11,1.0
1,167.892,0.625471,11.6175,504.140,0.293168,0.001107,62.1436,1.0,1.0,1.0,...,0.0,8279.1,14.35530,238.111,0.620147,0.667695,67.0974,60.0435,0.11,1.0
2,167.892,0.625471,11.6175,504.140,0.293168,0.001107,62.1436,1.0,1.0,1.0,...,0.0,8279.1,14.35530,238.111,0.620147,0.667695,67.0974,60.0435,0.11,1.0
3,167.892,0.625471,11.6175,504.140,0.293168,0.001107,62.1436,1.0,1.0,1.0,...,0.0,8279.1,14.35530,238.111,0.620147,0.667695,67.0974,60.0435,0.11,1.0
4,167.895,0.619473,11.6179,504.145,0.293009,0.001080,62.1472,1.0,1.0,1.0,...,0.0,8279.1,14.00090,264.802,0.620240,0.672394,67.1004,60.1557,0.11,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
971995,179.618,0.625473,11.9156,420.420,0.228225,1.903020,43.3472,2.0,1.0,1.0,...,0.0,8279.1,5.21471,783.722,0.536200,0.627771,64.5326,86.5871,1.47,-1.0
971996,179.618,0.625473,11.9156,420.420,0.228225,1.903020,43.3472,2.0,1.0,1.0,...,0.0,8279.1,5.21471,783.722,0.536200,0.627771,64.5326,86.5871,1.47,-1.0
971997,179.618,0.625473,11.9156,420.420,0.228225,1.903020,43.3472,2.0,1.0,1.0,...,0.0,8279.1,5.21471,783.722,0.536200,0.627771,64.5326,86.5871,1.47,-1.0
971998,179.618,0.625473,11.9156,420.420,0.228225,1.903020,43.3472,2.0,1.0,1.0,...,0.0,8279.1,5.21471,783.722,0.536200,0.627771,64.5326,86.5871,1.47,-1.0


In [9]:
df_normal=df[df['Attack LABLE (1:No Attack, -1:Attack)']==1]
df_attack=df[df['Attack LABLE (1:No Attack, -1:Attack)']==-1]

In [10]:
sensors = [3,9,25,43]
df_normal = df_normal.iloc[:, sensors]
df_attack=df_attack.iloc[:,sensors]

In [11]:
df_normal

,1_AIT_004_PV,1_MV_003_STATUS,2_FIC_401_CO,2_LS_101_AH
0,504.140,1.0,100.00000,0.0
1,504.140,1.0,100.00000,0.0
2,504.140,1.0,100.00000,0.0
3,504.140,1.0,100.00000,0.0
4,504.145,1.0,100.00000,0.0
...,...,...,...,...
971938,420.706,1.0,9.79881,0.0
971939,420.706,1.0,9.79881,0.0
971940,420.706,1.0,9.79881,0.0
971941,420.706,1.0,9.79881,0.0


In [14]:
def calc_width_height(v):
    width = 2/np.sqrt(v) 
    return width

    
def draw_ellipse(i,j,centers,widths,X):
    
    plt.xlim(centers[i]-widths[i]/2,centers[i]+widths[i]/2)
    plt.ylim(centers[j]-widths[j]/2,centers[j]+widths[j]/2)

    ellipse = Ellipse(xy=(centers[i], centers[j]), width=widths[i], height=widths[j])
    ellipse.fill = False
    ellipse.set_ec('red')
    ax.add_patch(ellipse)
    plt.scatter(x = X[::,i], y = X[::,j])
    

In [15]:
X = df_normal.iloc[:,2].values
X = hankel.fit(X,30,0.5)
X=X.T
kmeans,optimal_k = cluster.fit(X)
print(optimal_k)
radiis = []
weights=[]
centers=[]
clusters_R=[]
clusters_V=[]
# tsne=TSNE(n_components=2,perplexity=30)


for i in range(optimal_k):
    cluster_ = X[np.where(kmeans.labels_ == i)[0]]
    r = rank.fit(cluster_)
    clusters_R.append(r)
    print("r: "+str(r))
    U,Sigma,VT = np.linalg.svd(cluster_)
    V = VT.T
    clusters_V.append(V)
    cluster_ = np.matmul(cluster_,V[:,:r])
    # cluster_=tsne.fit_transform(cluster_)
    weight,center = meepc.fit(cluster_)
    weights.append(weight)
    centers.append(center)
    widths=calc_width_height(weight)
    var1=np.square(cluster_-center)
    var2=np.matmul(weight,var1.T)
    max_radius=math.sqrt(np.amax(var2))
    radiis.append(max_radius)


#     #plotting
# #     ax = plt.gca()
# #     ellipse = Ellipse(xy=(center[0],center[1]), width=widths[0], height=widths[1])
# #     ellipse.fill = False 
# #     ellipse.set_ec('red')
# #     ax.add_patch(ellipse)
# #     sns.scatterplot(x = cluster_[:,0], y = cluster_[:,1])
# # plt.show()
radiis



4
r: 110
r: 107
r: 388
r: 104


[1.001961862968689, 1.000004201219971, 1.0006529258885757, 1.00137407591873]

In [14]:
if math.isnan(max_radius):
        print('Rank: ',r)
        print('Cluster Shape: ',cluster.shape)
        print('CLuster: ',cluster)
        print('Weight Shape: ', weight.shape)
        print('Weight: ', weight)
        print('Center Shape: ', center.shape)
        print('Center: ', center)
        print('Var1: ', var1)
        print('Var2: ', var2)

# testing 

In [28]:
df_test=pd.read_csv('~/data/testdata.csv')
df_test

,1_AIT_001_PV,1_AIT_002_PV,1_AIT_003_PV,1_AIT_004_PV,1_AIT_005_PV,1_FIT_001_PV,1_LT_001_PV,1_MV_001_STATUS,1_MV_002_STATUS,1_MV_003_STATUS,...,3_AIT_001_PV,3_AIT_002_PV,3_AIT_003_PV,3_AIT_004_PV,3_AIT_005_PV,3_FIT_001_PV,3_LT_001_PV,LEAK_DIFF_PRESSURE,TOTAL_CONS_REQUIRED_FLOW,"Attack LABLE (1:No Attack, -1:Attack)"
0,161.129,0.721464,11.7842,463.716,0.267211,1.890010,48.9329,2.0,1.0,1.0,...,0.0,8279.1,5.95083,743.356,0.594601,0.001333,64.5137,74.2107,1.92,1.0
1,161.129,0.721464,11.7842,463.716,0.267211,1.890010,48.9329,2.0,1.0,1.0,...,0.0,8279.1,5.95083,743.356,0.594601,0.001333,64.5137,74.2107,1.92,1.0
2,161.129,0.721464,11.7842,463.716,0.267211,1.890010,48.9329,2.0,1.0,1.0,...,0.0,8279.1,5.95083,743.356,0.594601,0.001333,64.5137,74.2107,1.92,1.0
3,161.129,0.721464,11.7842,463.716,0.267211,1.890010,48.9329,2.0,1.0,1.0,...,0.0,8279.1,5.95083,743.356,0.594601,0.001333,64.5137,74.2107,1.92,1.0
4,161.129,0.721464,11.7842,463.716,0.267211,1.890010,48.9329,2.0,1.0,1.0,...,0.0,8279.1,5.95083,743.356,0.594601,0.001333,64.5137,74.2107,1.92,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248395,176.467,0.613474,11.9281,430.680,0.228327,0.001214,66.1851,1.0,1.0,1.0,...,0.0,8279.1,13.83640,370.386,0.729505,0.595032,64.3879,66.1173,1.41,1.0
248396,176.467,0.613474,11.9281,430.680,0.228327,0.001214,66.1851,1.0,1.0,1.0,...,0.0,8279.1,13.83640,370.386,0.729505,0.595032,64.3879,66.1173,1.41,1.0
248397,176.467,0.613474,11.9281,430.680,0.228327,0.001214,66.1851,1.0,1.0,1.0,...,0.0,8279.1,13.83640,370.386,0.729505,0.595032,64.3879,66.1173,1.41,1.0
248398,176.467,0.613474,11.9281,430.680,0.228327,0.001214,66.1851,1.0,1.0,1.0,...,0.0,8279.1,13.83640,370.386,0.729505,0.595032,64.3879,66.1173,1.41,1.0


In [29]:
y_actual=df_test.iloc[:,-1]
df_test_normal=df_test[df_test[ 'Attack LABLE (1:No Attack, -1:Attack)']==-1]
sensors = [3,9,25,43]
df_test = df_test_normal.iloc[:, sensors]
df_test

,1_AIT_004_PV,1_MV_003_STATUS,2_FIC_401_CO,2_LS_101_AH
241200,420.403,1.0,15.6107,0.0
241201,420.403,1.0,15.6107,0.0
241202,420.403,1.0,15.6107,0.0
241203,420.403,1.0,15.6107,0.0
241204,420.414,1.0,16.4946,0.0
...,...,...,...,...
247019,442.399,1.0,36.5606,0.0
247020,442.180,1.0,36.5166,0.0
247021,442.180,1.0,36.5166,0.0
247022,442.180,1.0,36.5166,0.0


In [30]:
y_actual.value_counts()

 1.0    246888
-1.0      1512
Name: Attack LABLE (1:No Attack, -1:Attack), dtype: int64

In [32]:
X_test=df_test.iloc[:,2].values
Xtest=hankel.fit(X_test,30,0.5)  #size -> 1800x274
Xtest=Xtest.T  #size -> 274x1800
print(Xtest.shape)
for i in range(optimal_k):
    Xtest_cluster=np.matmul(Xtest,clusters_V[i][:,:clusters_R[i]]) # size -> 274 x R
    var1=np.square(Xtest_cluster-centers[i])
    var2=np.matmul(weights[i],var1.T)
    widths=calc_width_height(weights[i])
    print("max radii",math.sqrt(np.amax(var2)))

(1, 1512)


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 1800 is different from 1512)

In [ ]:
X_twe = df_normal.iloc[:,2].values
X_twe = hankel.fit(X_twe,30,0.5)
print(X_twe.shape)

(1800, 1069)


In [ ]:
print(widths)

[1.16317836e+04 1.07125422e+04 1.01657036e+04 1.04247622e+04
 6.47384377e+03 4.62120997e+03 4.80540080e+03 4.81639727e+03
 2.23703115e+03 2.69198828e+03 1.66731242e+03 1.44290441e+03
 1.32032256e+03 1.20702758e+03 9.87513346e+02 1.11173506e+03
 1.30688286e+03 7.41116642e+02 1.06869518e+03 4.99332379e+02
 1.34293293e+03 5.51515714e+02 7.75660484e+02 5.42959563e+02
 3.15460934e+02 6.15609907e+02 2.88386002e+02 2.91832306e+02
 4.75629225e+02 2.74967444e+02 2.57016957e+02 1.90045588e+02
 1.72231750e+02 2.41651201e+02 1.64279060e+02 2.01082987e+02
 1.75632730e+02 1.53666239e+02 1.62311503e+02 1.69138905e+02
 1.68167488e+02 1.94465009e+02 1.13674700e+02 1.10683010e+02
 9.18429170e+01 9.89622843e+01 1.05455896e+02 8.30597619e+01
 7.54422869e+01 8.82215556e+01 7.56528696e+01 7.43443246e+01
 7.69083912e+01 6.22402568e+01 7.25901152e+01 6.72396996e+01
 6.77815058e+01 6.07322344e+01 6.27341293e+01 6.68115383e+01
 9.84497410e+01 4.80200170e+01 6.10517949e+01 5.33611912e+01
 5.27218807e+01 5.860947